Monthly Budget Code

In [3]:
'''Bespoke solution to the problem of reconciling 6 different excel 
spreadsheets maintained by dispersed teams and reconciling it with the 
databse that processes the actuals. This is then formatted for 
presentation back to the teams to track their budgets monthly. '''
        

import pandas as pd
import numpy as np


## Add write in B1 the budgets for each region. In addition, add column that shows
## The difference between projected and actuals. Fix True/ False column so that it is not false
## when recommended is blank or projected is 0. Also add a subtotal to the $ columns

def reconciliation(projections, actuals, month):

    ''' Takes two excel documens, proections and actuals and 
        Creates a third Excel workbook where each 
        region is into independent tabs'''
    
     def fromat_excel(workbook, worksheet, budget):
            
            '''Adds excel specific formatting to the document'''
        
            currency_format = workbook.add_format({'num_format': '$#,##0', 
                                                   'align'     : 'center'})         
            wrap_format     = workbook.add_format({'text_wrap' :  1}) 
            align_format    = workbook.add_format({'align'     : 'center'}) 
            format1         = workbook.add_format({'bg_color'  : '#FFC7CE',
                                                   'font_color': '#9C0006'})
            worksheet.set_column('A:A', 50)
            worksheet.set_column('B:B', 40)
            worksheet.set_column('C:C', 15, align_format)
            worksheet.set_column('D:D', 15, None, {'hidden': 1})
            worksheet.set_column('E:E', 15, currency_format)
            worksheet.set_column('F:F', 40)
            worksheet.set_column('G:G', 15, currency_format)
            worksheet.set_column('H:H', 15, currency_format, {'hidden': 1})
            worksheet.set_column('I:I', 15, None, {'hidden': 1})
            worksheet.set_column('J:J', 10)
            worksheet.set_column('K:K', 15, currency_format)

            worksheet.conditional_format('J1:J{}'.format(str(df_shape[0] + 1)), {'type': 'cell',
                                                                                 'criteria': '=',
                                                                                 'value': False,
                                                                                 'format': format1}) 

            table_end = ('K' + str(df_shape[0] + 2))
            table_range = 'A1:{}'.format(table_end)
            worksheet.add_table(table_range, {  'first_column': True,
                                                'total_row'   : 1,
                                            'columns': [{'header': 'Organization'},
                                                        {'header': 'Project 2019'},
                                                        {'header': 'Staff'},
                                                        {'header': 'Region'},
                                                        {'header': 'Projected',     'total_function': 'sum'},
                                                        {'header': 'Project 2020'},
                                                        {'header': 'Recommended',   'total_function': 'sum'},
                                                        {'header': 'Grant Ammount', 'total_function': 'sum'},
                                                        {'header': 'Fund'},
                                                        {'header': 'On Target?'},
                                                        {'header': 'Difference',    'total_function': 'sum'}]})
            worksheet.write_string(int(df_shape[0] + 6), 9, 'Projections')
            worksheet.write_string(int(df_shape[0] + 7), 9, 'Over Projections')
            worksheet.write_string(int(df_shape[0] + 8), 9, 'Adjusted Projections')
            worksheet.write_string(int(df_shape[0] + 9), 9, budget)
            worksheet.write_number(int(df_shape[0] + 10), 10, )
            worksheet.write_string(int(df_shape[0] + 10), 9, 'BUDGET')
            worksheet.write_string(int(df_shape[0] + 11), 9, 'Remaining')
    
    projections = pd.read_excel(projections,
                                sheet_name = 1, 
                                usecols = 'A:B,D,G, H').rename(columns={'Organization': 'Name', 
                                                                        '2020 Expected': 'projected'})
                                
    actuals = pd.read_excel(actuals, 
                            usecols ='C:F,H').rename( columns = {'Recommended Amount': 'recommended'})

    book = pd.merge(projections, 
                    actuals, 
                    how      = 'outer', 
                    left_on  = 'Name',
                    right_on = 'Name').set_index('Name')
                           
    book['on_target']  = np.where(book.projected == book.recommended, 
                                  True, 
                                  False)
    
    book['difference'] = book.recommended - book.projected      
    
    # Below is region : [short_id, budget #]
    regions = { 'Region_1' : ['reg_1', 0],
                'Region_2' : ['reg_2', 0],                        
                'Region_3' : ['reg_3', 0], 
                'Region_4' : ['reg_4', 0], 
                'Region_5' : ['reg_5', 0],
                'Region_6' : ['reg_6', 0]}

    month = pd.ExcelWriter('{}_reconciliation.xlsx'.format(month),  
                        engine ='xlsxwriter')
   
    for region in regions.items():
        key   = region[0]
        value = region[1][0]
        value = book[(book.Region == key) | 
                     (book.Fund == key)
                                            ]
        df_shape = value.shape     
    
        value.to_excel(month, sheet_name = key)
        workbook  = month.book
        worksheet = month.sheets[key]
        
        format_excel(workbook, worksheet, budgets.get(key)[1])

        
    month.close()


reconciliation('projections.xlsx', 'actuals.xlsx', 'october')